In [1]:
import itertools
import concurrent.futures
from scipy.spatial.distance import cdist
import time
import heapq

def read_tsp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    start_index = None
    for i, line in enumerate(lines):
        if line.strip() == "NODE_COORD_SECTION":
            start_index = i + 1
            break

    if start_index is None:
        raise ValueError("Invalid TSP file format. Could not find NODE_COORD_SECTION.")

    tsp_graph = []
    for line in lines[start_index:]:
        line = line.strip()
        if line == "EOF":
            break
        tsp_graph.append(tuple(map(float, line.split()[1:])))

    coords = [coord for coord in tsp_graph]
    distance_matrix = cdist(coords, coords, 'euclidean')
    return tsp_graph, distance_matrix

def explore_path(params):
    node, path, cost, distances = params
    num_nodes = len(distances)
    unvisited = [n for n in range(1, num_nodes + 1) if n not in path]
    new_beam = []

    for candidate in unvisited:
        new_path = path + [candidate]
        new_cost = cost + distances[node - 1][candidate - 1]
        new_beam.append((candidate, new_path, new_cost))

    return new_beam

def beam_search_tsp_threaded(distances, beam_width):
    num_nodes = len(distances)
    initial_node = 1

    beam = [(initial_node, [initial_node], 0)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for _ in range(num_nodes - 1):
            new_beam = []

            params = [(node, path, cost, distances) for node, path, cost in beam]
            results = executor.map(explore_path, params)

            for result in results:
                new_beam.extend(result)

            beam =heapq.nsmallest(beam_width, new_beam, key=lambda x:x[2])
    final_path = beam[0][1]
    final_cost = beam[0][2]
    final_path.append(final_path[0])
    final_cost += distances[final_path[-2]-1][final_path[0]-1]

    return final_path, final_cost

file_path = "berlin52.tsp"
graph, distances = read_tsp_file(file_path)

beam_width = 100

start_time = time.perf_counter()
path, cost = beam_search_tsp_threaded(distances, beam_width)
end_time = time.perf_counter()
total_time = end_time - start_time

print('Optimal Cost:', cost)
print(total_time)



Optimal Cost: 9851.055028314973
0.49440059999999164
